In [1]:
import pandas as pd
import pymysql

In [2]:
csv_files = ["ASTC.csv",
             "HRTC.csv",
             "JKSRTC.csv",
             "KSRTC.csv",
             "KTCL.csv",
             "NBSTC.csv",
             "PEPSU.csv",
             "SBSTC.csv",
             "TSRTC.csv",
             "UPSRTC.csv"]

In [3]:
# Read each CSV file into a DataFrame and store it in a list
df_list = [pd.read_csv(file) for file in csv_files]

In [4]:
# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

In [5]:
# Clean up DataFrame
combined_df['Price'] = combined_df['Price'].str.replace('INR ', '', regex=False)
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HI\AppData\Local\Temp\ipykernel_11788\2157597570.py:3: SyntaxWarning: invalid escape sequence '\d'
  combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract('(\d+)')


In [6]:
# Drop null values
combined_df = combined_df.dropna()


In [7]:
# MySQL connection
myconnection = pymysql.connect(
    host="localhost", 
    user="root", 
    passwd="Iyyappan1*", 
    database="redbus"
)


In [8]:
combined_df.dtypes

Route_Name            object
Route_Link            object
Bus_Name              object
Bus_Type              object
Departing_Time        object
Duration              object
Reaching_Time         object
Star_Rating          float64
Price                 object
Seat_Availability     object
dtype: object

In [9]:
# Prepare column definitions for MySQL
a = ",".join(f"{i} {j}" for i, j in zip(combined_df.columns, combined_df.dtypes)).replace("float64", "float").replace("object", "text")


In [10]:
a

'Route_Name text,Route_Link text,Bus_Name text,Bus_Type text,Departing_Time text,Duration text,Reaching_Time text,Star_Rating float,Price text,Seat_Availability text'

In [12]:
combined_df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:15,05h 45m,03:00,3.7,450,17
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:45,05h 30m,03:15,3.5,450,13
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,DESTINY TRAVELS,NON A/C Seater (2+1),23:00,05h 15m,04:15,1.3,488.78,22
3,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,WARISPIYA TRAVELS,Bharat Benz A/C Seater (2+2),16:15,03h 25m,19:40,4.2,360,26
4,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...,Kalpana Travels,AC Seater (2+2),16:50,03h 25m,20:15,4.6,360,29
...,...,...,...,...,...,...,...,...,...,...
1624,Agra to Lucknow,https://www.redbus.in/bus-tickets/agra-to-lucknow,Anshi Raj Shree Travels,A/C Seater/Sleeper (2+1),23:59,05h 16m,05:15,3.4,854,21
1625,Agra to Lucknow,https://www.redbus.in/bus-tickets/agra-to-lucknow,Bhagwati Travel Agency,AC Sleeper (2+1),21:20,05h 00m,02:20,3.3,990,36
1626,Agra to Lucknow,https://www.redbus.in/bus-tickets/agra-to-lucknow,RS YADAV SMART BUS,VE A/C Seater / Sleeper (2+1),23:55,05h 20m,05:15,3.9,599,24
1627,Agra to Lucknow,https://www.redbus.in/bus-tickets/agra-to-lucknow,Anshi Raj Shree Travels,Bharat Benz A/C Sleeper (2+1),22:15,05h 15m,03:30,1.7,1139,12


In [13]:
# Create table
# Assuming 'a' contains the column definitions for the table excluding 'id'
table_name = "bus_routes"
myconnection.cursor().execute(f"CREATE TABLE IF NOT EXISTS {table_name} (id INT AUTO_INCREMENT PRIMARY KEY, {a})")

# Get the column names (excluding 'id') from 'a'
column_names = [col.split()[0] for col in a.split(',')] 

# Prepare the query string for insertion
columns_string = ", ".join(column_names)
placeholders = ", ".join(["%s"] * len(column_names))  # Placeholder for parameterized query

# Insert data into MySQL
query = f"INSERT INTO {table_name} ({columns_string}) VALUES ({placeholders})"
cursor = myconnection.cursor()

for _, row in combined_df.iterrows():
    cursor.execute(query, tuple(row))  # Using a parameterized query for safety

myconnection.commit()

# Close the MySQL connection
myconnection.close()

In [14]:
columns_string

'Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Availability'

In [15]:
placeholders

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s'